In [1]:
import pandas as pd
import numpy as np

import math
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
# 수비이닝 형변환 함수

def inning_change(value):
    
    tmp = value.split(' ')
    
    if len(tmp) == 2:
        if tmp[1] == '1/3':
            output = int(tmp[0]) + 1/3
        elif tmp[1] == '2/3':
            output = int(tmp[0]) + 2/3
            
    else: 
        if '1/3' == tmp[0]:
            output = 1/3
        elif '2/3' == tmp[0]:
            output = 2/3
        else:
            output = int(tmp[0])
        
    return output

In [3]:
#연도별 반복(2002~2020)
for YEAR in list(np.arange(2002, 2021)):
    
    #CSV파일 불러오기
    df = pd.read_csv('선수데이터/수비/수비정보_팀전체({}).csv'.format(YEAR), engine = 'python', encoding = 'utf-8-sig')

    #수비이닝 분수에서 소수로 변환
    for i in list(np.arange(0,len(df))):
        df['수비이닝'][i] = inning_change(df['수비이닝'][i])
    
    #포지션별 이닝수를 담을 새로운 데이터프레임 생성
    inning = df[{'선수명', 'ID', '팀명'}]
    inning[['포수', '투수', '1루수', '2루수', '3루수', '유격수', '외야수']] = 0


    #첫행부터 마지막행까지, df의 포지션값에 해당하는 inning의 포지션에 df의 수비정보값 저장
    for i in list(np.arange(0,len(inning))):
        if df['포지션'][i] == '포수':
            inning['포수'][i] = df['수비이닝'][i]

        elif df['포지션'][i] == '투수':
            inning['투수'][i] = df['수비이닝'][i]

        elif df['포지션'][i] == '1루수':
            inning['1루수'][i] = df['수비이닝'][i]

        elif df['포지션'][i] == '2루수':
            inning['2루수'][i] = df['수비이닝'][i]

        elif df['포지션'][i] == '3루수':
            inning['3루수'][i] = df['수비이닝'][i]

        elif df['포지션'][i] == '유격수':
            inning['유격수'][i] = df['수비이닝'][i]

        elif df['포지션'][i] == '좌익수' or df['포지션'][i] == '중견수' or df['포지션'][i] == '우익수':
            inning['외야수'][i] = df['수비이닝'][i]

    #inning을 ID값으로 GROUP BY, 각 포지션별 수비이닝을 구해 entropy에 저장, 포지션 엔트로피열 추가
    entropy = inning.groupby('ID')[['포수', '투수', '1루수', '2루수', '3루수', '유격수', '외야수']].sum().reset_index()
    entropy['통산이닝수'] = 0
    entropy['포지션 엔트로피'] = 0.000000
    
    #각 포지션별 이닝수 합을 통산이닝수로 둠
    for i in list(np.arange(0,len(entropy))):
        entropy['통산이닝수'][i] = sum(entropy.iloc[i,1:8])
    
    #각 포지션별 이닝수를 통산이닝수로 나눠 해당 포지션 출전비율로 덮어씌움
    entropy['포수'] = entropy['포수']/entropy['통산이닝수']
    entropy['투수'] = entropy['투수']/entropy['통산이닝수']
    entropy['1루수'] = entropy['1루수']/entropy['통산이닝수']
    entropy['2루수'] = entropy['2루수']/entropy['통산이닝수']
    entropy['3루수'] = entropy['3루수']/entropy['통산이닝수']
    entropy['유격수'] = entropy['유격수']/entropy['통산이닝수']
    entropy['외야수'] = entropy['외야수']/entropy['통산이닝수']

    #피치 엔트로피에서는 로그의 밑이 10인 Hartley 엔트로피를 사용했지만, 포지션 엔트로피에서는 밑이 2인 Shannon 엔트로피를 사용함
    #각 포지션의 포지션 엔트로피 값(- 출전비율 * 출전비율의 로그값)을 덮어씌움
    for j in list(np.arange(0,len(entropy))):
        for position in ['포수', '투수', '1루수', '2루수', '3루수', '유격수', '외야수']:
            try:
                entropy[position][j] = -entropy[position][j] * math.log2(entropy[position][j])
            except ValueError:
                entropy[position][j] = 0
    
    #각 포지션의 포지션 엔트로피 값을 합해 선수의 포지션 엔트로피 값을 도출
    for i in list(np.arange(0,len(entropy))):
        entropy['포지션 엔트로피'][i] = sum(entropy.iloc[i,1:8])

    #해당 연도 추가
    entropy['연도'] = YEAR

    #기타 계산용 열 제외
    result = entropy[['ID', '연도', '포지션 엔트로피']]

    #result와 중복을 제외한 df을 ID를 기준으로 merge, 선수명과 팀명을 가져오기 위함
    result_merge = pd.merge(result, df.drop_duplicates('ID'), how='inner',on='ID', )
    result_merge = result_merge[['ID', '선수명', '팀명', '연도', '포지션 엔트로피']]

    #엑셀파일로 저장
    result_merge.to_csv('포지션엔트로피데이터/연도별데이터/포지션엔트로피({}).csv'.format(YEAR), index=False)

In [4]:
#한개 파일로 합치기
last = pd.DataFrame()

# 년도정의 (2002~2020)
for YEAR in list(np.arange(2002, 2021)):
    #csv 데이터 프레임으로 불러오기(경로는 내 기준, 파일 위치에 따라 수정필요)
    read = pd.read_csv('포지션엔트로피데이터/연도별데이터/포지션엔트로피({}).csv'.format(YEAR))
    last = pd.concat([last, read])
            
#엑셀파일로 저장
last.to_csv('포지션엔트로피데이터/포지션엔트로피.csv', index=False)